Shamir Secret Sharing:

- Enter the secret, s, and the minimum number of recipients, t, of n shares.

- Let p > max(s, n), with $\mathbb{F}_p$ as our finite field. 

- Generate random coefficients for a polynomial of degree t-1, f(x), with the y-intercept as the secret text.

- Compute n points on the polynomial, (x, f(x)), with $x \neq 0$.

- Give each participant a unique point.

- To reconstruct the secret, use Lagrange interpolation formula with t many points of interpolation, and evaluate it at f(0) mod p.

In [3]:
import pandas as pd 
import sagemath as sg 
import random as rd
import numpy as np
import plotly.express as px
from random import randint
from math import gcd
import gmpy2
from gmpy2 import mpz, is_prime, gcdext, invert
import sympy as sp
from sympy import isprime, factorint, mod_inverse
import numpy.polynomial.polynomial as poly

Encoding

In [11]:
secret = 17
min_reps = 7
n = 100

p = max(secret, n) + 1
def getPrime(p):
    k = True

    while k == True:
        if isprime(p) == True:
            k = False
        else:
            p = p +1
        
    return p

prime = getPrime(p)
print(f'We are using Z mod {prime} as our finite field.')


def polynomialMaker(t,prime):
    polylist = [secret]
    for i in range(t-1):
        polylist = polylist + [randint(0,prime)]
    eq = poly.Polynomial(polylist)
    return eq

encodingPoly = polynomialMaker(min_reps,prime)
print('\nPolynomial:')
display(encodingPoly)


def splitter(eq, n, prime):
    points = pd.DataFrame(columns=['X', 'f(X)'])
    list = []
    for i in range(n):
        k = randint(1,n)
        if k in list:
            continue
        else:
            list = list + [k]
            points.loc[i, 'X'
                       ] = k
            num = eq(k)
            points.loc[i, 'f(X)'] = num 
    
    return points

ciphertext = splitter(encodingPoly, n, prime)
display(ciphertext)

We are using Z mod 101 as our finite field.

Polynomial:


Polynomial([17.,  7., 11., 34., 13., 42., 35.], domain=[-1,  1], window=[-1,  1], symbol='x')

,X,f(X)
0,63,2230218246329.0
1,78,8003744746079.0
2,82,10796550813339.0
3,68,3521715801629.0
5,16,632236929.0
...,...,...
91,25,8960694567.0
92,70,4188628246407.0
96,49,496387981743.0
97,96,27740113505969.0


Decoding:

In [12]:
def modinv(a, p):
    return pow(a, p - 2, p)

def lagrange_symbolic(df, p):
    x = sp.Symbol('x')
    xs = [int(v) for v in df['X']]
    ys = [int(v) for v in df['f(X)']]
    n = len(xs)

    poly = 0
    for i in range(n):
        xi, yi = xs[i], ys[i]
        term = yi
        numerator = 1
        denominator = 1
        Li = 1
        for j in range(n):
            if i != j:
                xj = xs[j]
                numerator *= (x - xj)
                denominator *= (xi - xj) % p
        Li = numerator * modinv(denominator, p)
        poly += term * Li

    # Expand and reduce modulo p
    poly = sp.expand(poly)
    coeffs = sp.Poly(poly, x, modulus=p).all_coeffs()  # highest degree first

    # Convert to ascending order
    return [int(c) for c in reversed(coeffs)]

shift_factor = 0

decodingPoly = lagrange_symbolic(ciphertext.head(min_reps+shift_factor), prime)
eq = poly.Polynomial(decodingPoly)
print(f'Lagrange Polynomial is {eq}')
print(f'\nSafetext was {(eq(0)%prime)}')


Lagrange Polynomial is 17.0 + 7.0 x + 11.0 x**2 + 34.0 x**3 + 13.0 x**4 + 42.0 x**5 + 35.0 x**6

Safetext was 17.0
